# Creating a Medical triage workflow with TigerGraph
<ol>


# Set Up


In [2]:
!python --version
!pip install pyTigerGraph

Python 2.7.16


# Exploring pyTigerGraph

In [4]:
!pip install pyTigerGraph

In [6]:
import pyTigerGraph as tg # Importing

# Create a Connection
conn = tg.TigerGraphConnection(host="https://healthnet.i.tgcloud.io", password="tigergraph", gsqlVersion="3.0.5", useCert=True)

In [7]:
# Sets up connection with GSQL
print(conn.gsql('ls', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
Edge Types:

Graphs:
Jobs:


JSON API version: v2
Syntax version: v2



In [21]:
# Create Edges (published and references) and Vertices (source, article and event)
print(conn.gsql('''
  CREATE VERTEX condition(PRIMARY_ID name STRING, desciption STRING, icd_code STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX symptom(PRIMARY_ID name STRING, severity INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX precaution(PRIMARY_ID id STRING, description STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX specialist(PRIMARY_ID name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"

  CREATE DIRECTED EDGE has_symptom(FROM condition, TO symptom) WITH REVERSE_EDGE="reverse_has_symptom"
  CREATE DIRECTED EDGE has_precaution(FROM condition, TO precaution) WITH REVERSE_EDGE="reverse_has_precaution"
  CREATE DIRECTED EDGE refer_to(FROM condition, TO specialist) WITH REVERSE_EDGE="reverse_refer_to"

  ''', options=[]))

print(conn.gsql('''CREATE GRAPH healthnet(condition, symptom, precaution, specialist, has_symptom, has_precaution, refer_to)''', options=[])) # Create the Graph



Successfully created vertex types: [condition].
Successfully created vertex types: [symptom].
Successfully created vertex types: [precaution].
Successfully created vertex types: [specialist].
Successfully created edge types: [has_symptom].
Successfully created reverse edge types: [reverse_has_symptom].
Successfully created edge types: [has_precaution].
Successfully created reverse edge types: [reverse_has_precaution].
Successfully created edge types: [refer_to].
Successfully created reverse edge types: [reverse_refer_to].
Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 30.694 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.098 seconds
The graph healthnet is created.


In [19]:
# Edit the Connect Credentials for Loading Data
conn.graphname = "healthnet"
conn.apiToken = conn.getToken(conn.createSecret())

Now we can import the CSV media rating into the graph.

###  What is a ICD-10 diagnosis code?
Used for medical claim reporting in all healthcare settings, ICD-10-CM is a standardized classification system of diagnosis codes that represent conditions and diseases, related health problems, abnormal findings, signs and symptoms, injuries, external causes of injuries and diseases, and social circumstances

Jobs for importing CSV into the graph


In [15]:
print(conn.gsql('''
CREATE LOADING JOB load_job_specialists_csv_1650501210039 FOR GRAPH healthnet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX specialist VALUES($0) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

CREATE LOADING JOB load_job_symptom_severity_csv_1650501255514 FOR GRAPH healthnet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX symptom VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

CREATE LOADING JOB load_job_disease_precaution_csv_1650501525366 FOR GRAPH healthnet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE has_precaution VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE has_precaution VALUES($0, $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE has_precaution VALUES($0, $3) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE has_precaution VALUES($0, $4) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

CREATE LOADING JOB load_job_disease_symptoms_csv_1650501655558 FOR GRAPH healthnet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE has_symptom VALUES($0, $1) USING SEPARATOR=",", HEADER="false", EOL="\n";
      LOAD MyDataSource TO EDGE has_symptom VALUES($0, $2) USING SEPARATOR=",", HEADER="false", EOL="\n";
      LOAD MyDataSource TO EDGE has_symptom VALUES($0, $3) USING SEPARATOR=",", HEADER="false", EOL="\n";
      LOAD MyDataSource TO EDGE has_symptom VALUES($0, $4) USING SEPARATOR=",", HEADER="false", EOL="\n";
      LOAD MyDataSource TO EDGE has_symptom VALUES($0, $5) USING SEPARATOR=",", HEADER="false", EOL="\n";
      LOAD MyDataSource TO EDGE has_symptom VALUES($0, $5) USING SEPARATOR=",", HEADER="false", EOL="\n";
    }

CREATE LOADING JOB load_job_disease_description_csv_1650503394334 FOR GRAPH healthnet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX condition VALUES($0, $1, _) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

 ''', options=[]))



Successfully created loading jobs: [load_job_specialists_csv_1650482322027].
Successfully created loading jobs: [load_job_Symptom_severity_csv_1650482478169].


## Peek into Data Load


import pandas as pd

df = pd.read_csv('data/conditions.csv')
print(df.head(0))
print('######')
df = pd.read_csv('data/specialists.csv')
print(df.head(0))
print('######')
df = pd.read_csv('data/disease-symptoms.csv')
print(df.head(0))
print('######')
df = pd.read_csv('data/disease_description.csv')
print(df.head(0))
print('######')
df = pd.read_csv('data/symptom-severity.csv')
print(df.head(0))
print('######')
df = pd.read_csv('data/disease_precaution.csv')
print(df.head(0))
print('######')

### Clear the Whole Graph
DANGER ZONE

In [20]:
conn.gsql('''
USE GLOBAL
DROP ALL
''')

'Dropping all, about 1 minute ...\nAbort all active loading jobs\nTry to abort all loading jobs on graph healthnet, it may take a while ...\n[ABORT_SUCCESS] No active Loading Job to abort.\nResetting GPE...\nSuccessfully reset GPE and GSE\nStopping GPE GSE\nSuccessfully stopped GPE GSE in 0.009 seconds\nClearing graph store...\nSuccessfully cleared graph store\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.094 seconds\nEverything is dropped.'

### Load data

In [ ]:
import json

specialist_file = "data/specialists.csv"
res = conn.uploadFile(specialist_file, fileTag='MyDataSource', jobName='load_job_specialists_csv_1650501210039')
print(json.dumps(res, indent=2))


symptoms_file = "data/symptom_severity.csv"
res = conn.uploadFile(symptoms_file, fileTag='MyDataSource', jobName='load_job_symptom_severity_csv_1650501255514')
print(json.dumps(res, indent=2))


precaution_file = "data/disease_precaution.csv"
res = conn.uploadFile(precaution_file, fileTag='MyDataSource', jobName='load_job_disease_precaution_csv_1650501525366')
print(json.dumps(res, indent=2))


disease_symptoms_file = "data/disease_symptoms.csv"
res = conn.uploadFile(disease_symptoms_file, fileTag='MyDataSource', jobName='load_job_disease_symptoms_csv_1650501655558')
print(json.dumps(res, indent=2))


disease_description_file = "data/disease_description.csv"
res = conn.uploadFile(disease_description_file, fileTag='MyDataSource', jobName='load_job_disease_description_csv_1650503394334')
print(json.dumps(res, indent=2))

